## **Random Forest implementation using NumPy**

**Import libraries**

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time

**Model parameters**

In [2]:
no_of_samples = 10000
no_of_columns = 16
no_of_buckets  = 10
gini_thresh = 0.03
min_obs = 20
row_sub_sampling = 0.8
no_of_trees = 100
no_of_records_in_train = int (no_of_samples * 0.7)
no_of_features_per_split = 4

**Create dataset**

In [3]:
np.random.seed(1)
normal_sample = np.random.normal(loc=0, scale=1,size= (no_of_samples,no_of_columns))

normal_sample_class = np.where( np.logical_or( (normal_sample[:,4].reshape(no_of_samples,1) + normal_sample[:,2].reshape(no_of_samples,1)) > 1.5,
                                             (normal_sample[:,0].reshape(no_of_samples,1) + normal_sample[:,1].reshape(no_of_samples,1)) < -0.33), np.ones((no_of_samples,1 ), dtype = 'int64'), np.zeros((no_of_samples,1 ), dtype = 'int64')  )


**Model definition**

In [4]:
def calculate_split_vector(tmp_min_max_vec):
    return np.linspace(start = tmp_min_max_vec[0], stop = tmp_min_max_vec[1], num=no_of_buckets)[1:-1]
def apply_splits(tmp_col, tmp_df, tmp_df_class):
    length = tmp_df.shape[0]
    sum_of_one_class = np.sum(tmp_df_class)
    prop_zer_class = (length - sum_of_one_class)/length
    prop_one_class = sum_of_one_class/length
    ge_than_split = np.greater_equal(tmp_df, tmp_col)
    ge_than_split_sum = np.sum(ge_than_split, axis = 0)
    cross_table_11 = np.sum(ge_than_split[ tmp_df_class[:,0] ], axis=0)
    cross_table_10 = sum_of_one_class - cross_table_11
    cross_table_01 = np.sum(ge_than_split[ tmp_df_class[:,0] !=True], axis=0)
    cross_table_00 = (length - sum_of_one_class) - cross_table_01
    gini_zero_class_pred = 1 - ((np.power(cross_table_10, 2 ) + np.power(cross_table_00, 2 ) ) / np.power(length - ge_than_split_sum  ,2))
    gini_one_class_pred = 1 - ((np.power(cross_table_11, 2 ) + np.power(cross_table_01, 2 ) ) / np.power(ge_than_split_sum  ,2))
    gini_both_classes_stack = np.vstack( (gini_zero_class_pred  *((length - ge_than_split_sum)/length ), gini_one_class_pred *(ge_than_split_sum/length ) ) )
    gini_both_classes = np.sum( gini_both_classes_stack , axis = 0)
    
    best_pred_class = np.argmin(gini_both_classes_stack, axis = 0)#.reshape(-1,1)    
    original_classes = np.vstack((np.argmax(np.vstack((cross_table_00, cross_table_10 )), axis = 0 ),
           np.argmax(np.vstack((cross_table_01, cross_table_11 )), axis = 0 ))).T
    classes_to_return = np.where(best_pred_class == 0 , original_classes[:,0], original_classes[:, 1] )
    best_col_selected = np.argmin(gini_both_classes)
    return [best_col_selected,  gini_both_classes[ best_col_selected], tmp_col[ best_col_selected] ,best_pred_class[best_col_selected], classes_to_return[best_col_selected] ] #, best_pred_class, classes_to_return

def calc_gini( tmp_df, axes):
    return np.sum(tmp_df[axes,:,:], axis=1)
    


def calculate_split(subset_indicies, tmp_train_set, tmp_class_arr):
    subset_indicies_len = subset_indicies.shape[0]
    features_draw_for_split = np.random.choice(range(no_of_columns), no_of_features_per_split, replace = False)#.reshape(-1,1)
    tmp_df = (tmp_train_set[subset_indicies, :])[:, features_draw_for_split]
    tmp_df_class = tmp_class_arr[subset_indicies, : ]
    tmp_df_class_bool = tmp_df_class == 1
    tmp_min_array = np.min(tmp_df, axis = 0)
    tmp_max_array = np.max(tmp_df, axis = 0)
    tmp_splits_array = np.apply_along_axis(calculate_split_vector, 0, np.vstack((tmp_min_array, tmp_max_array)))
    tmp_splits_applied= np.apply_along_axis(apply_splits, 1, tmp_splits_array, tmp_df, tmp_df_class_bool) #, returned_l_r, returned_classes 
    best_gini_location = np.unravel_index(np.argmin(tmp_splits_applied, axis=None), tmp_splits_applied.shape)                    
    best_col_best_split = tmp_splits_applied[np.argmin(tmp_splits_applied[:,1])]
    tmp_sub_column_index = int(best_col_best_split[0])
    best_col_best_split[0] = features_draw_for_split[tmp_sub_column_index]
    return best_col_best_split , subset_indicies[(tmp_df)[ :,  tmp_sub_column_index   ] >= best_col_best_split[2]], subset_indicies[(tmp_df)[ :,  tmp_sub_column_index    ] < best_col_best_split[2]]




class Tree:
    def __init__(self, tmpIndicies, tmp_train, tmp_classes):
        #start_time = datetime.now()
        split_matrix_1, right_ind_1, left_ind_1 = calculate_split(tmpIndicies, tmp_train, tmp_classes)
        self.split_column = int(split_matrix_1[0])
        self.split_value = (split_matrix_1[2])
        self.right_class = return_right_class(split_matrix_1)
        if under_gini_threshold(split_matrix_1):
            self.leftNode = None
            self.rightNode = None
        else:
            if (len(left_ind_1) < min_obs) or check_two_classes(left_ind_1, tmp_classes):
                self.leftNode = None
            else:
                self.leftNode = Tree(left_ind_1, tmp_train, tmp_classes)
            if (len(right_ind_1) < min_obs) or check_two_classes (right_ind_1, tmp_classes) :
                self.rightNode = None
            else:
                self.rightNode = Tree(right_ind_1, tmp_train, tmp_classes)
        #print(str(datetime.now() - start_time))
                
            
    def predict(self, record_to_predict):
        if record_to_predict[self.split_column] >= self.split_value:
            if self.rightNode:
                return self.rightNode.predict(record_to_predict)               
            else:
                return self.right_class
        else:
            if self.leftNode:
                return self.leftNode.predict(record_to_predict)
            else:
                return 1 - self.right_class
            
        
        
    def insert(self, valueToAdd):
        if self.value != valueToAdd:
            if self.value > valueToAdd:
                if self.leftNode:
                    self.leftNode.insert(valueToAdd)
                else:
                    self.leftNode = Node(valueToAdd)
            else:
                if self.rightNode:
                    self.rightNode.insert(valueToAdd)
                else:
                    self.rightNode = Node(valueToAdd)                    
            
        
    def showTree(self, level, l_or_r = 'root'):
        print ('level: {} {} column {}, split value {}'.format(level, l_or_r,  self.split_column, self.split_value))
        if self.leftNode:
            self.leftNode.showTree(level + 1, 'left')
        if self.rightNode:
            self.rightNode.showTree(level + 1, 'right')

def under_gini_threshold(tmp_mtrx):
    return tmp_mtrx[1] <= gini_thresh

def return_right_class(tmp_mtrx):
    if tmp_mtrx[3] > 0.5:
        return tmp_mtrx[4]
    else:
        return 1 - tmp_mtrx[4]

def check_two_classes(tmp_ind, tmp_df):
    return (np.unique(tmp_df[tmp_ind])).size == 1    




class TreeModel:
    def __init__(self):
        self.listOfTrees = []
    def fit(self, tmp_train_data, tmp_classes):
        np.random.seed(1)
        self.listOfTrees = [ Tree(np.random.choice(range(tmp_train_data.shape[0]), size=int(row_sub_sampling * tmp_train_data.shape[0] ), replace=True),  tmp_train_data, tmp_classes) for _ in range(no_of_trees) ]
    def predict(self, tmp_test_set):
        results = np.zeros((tmp_test_set.shape[0], no_of_trees))
        for tmp_tree_cnt in range(len(self.listOfTrees)):
            results[:, tmp_tree_cnt] = np.apply_along_axis(self.listOfTrees[tmp_tree_cnt].predict, 1, tmp_test_set)#.reshape(tmp_test_set.shape[0],1)   
        return np.where(np.mean(results, axis=1)>0.5,1,0)#.shape
        


**Train and predict**

In [5]:

train_set = normal_sample[:no_of_records_in_train]
train_set_class = normal_sample_class[:no_of_records_in_train]

start_time = datetime.now()

newModel = TreeModel()
newModel.fit(train_set, train_set_class)

print(str(datetime.now() - start_time))

pd.crosstab(newModel.predict(normal_sample[no_of_records_in_train:]), normal_sample_class[no_of_records_in_train:,0], rownames = ['predictions'], colnames = ['values'])


0:00:39.110233


values,0,1
predictions,,
0,1499,42
1,43,1416
